https://coin-or.github.io/pulp/CaseStudies/a_set_partitioning_problem.html

In [1]:
import pandas as pd

from pulp import LpProblem, LpStatus, LpVariable, lpSum, value, allcombinations
from pulp import LpBinary, LpMinimize, LpMaximize

In [2]:
MAX_NUMBER_OF_TABLES = 8

MIN_TABLE_SIZE = 2
MAX_TABLE_SIZE = 4

GUESTS = "A B C D E F G H I J K L M N O P Q R S T U V W X Y Z".split()

In [3]:
# Create list of all possible tables -- with at least two guests to a table
GUEST_COMBINATIONS = [
    tuple(combo)
    for combo in allcombinations(GUESTS, MAX_TABLE_SIZE)
    if len(combo) >= MIN_TABLE_SIZE
]

In [4]:
# Create a binary variable to state that a table setting is used
tables = LpVariable.dicts(
    "tables", GUEST_COMBINATIONS, lowBound=0, upBound=1, cat="Integer"
)

In [5]:
model = LpProblem("Wedding_Seating_Model", LpMaximize)

In [6]:
def group_diversity(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    value = 0
    for combo in allcombinations(table, 2):
        value += abs(ord(combo[0]) - ord(combo[-1]))
    return value

In [7]:
# ---
# Objective function
# ---
model += lpSum([group_diversity(combo) * tables[combo] for combo in GUEST_COMBINATIONS])

# ---
# Constraints
# ---

# Specify the maximum number of tables
model += (
    lpSum([tables[combo] for combo in GUEST_COMBINATIONS]) <= MAX_NUMBER_OF_TABLES,
    "Maximum number of tables",
)

# A guest must seated at one and only one table
for guest in GUESTS:
    model += (
        lpSum([tables[combo] for combo in GUEST_COMBINATIONS if guest in combo]) == 1,
        f"Must seat {guest}",
    )

In [8]:
model.solve()

LpStatus[model.status]

'Optimal'

In [9]:
value(model.objective)

409.0

In [10]:
print("The choosen tables are out of a total of %s:" % len(GUEST_COMBINATIONS))
for guests in GUEST_COMBINATIONS:
    if tables[guests].varValue == 1.0:
        print(guests)

The choosen tables are out of a total of 17875:
('G', 'Q')
('A', 'J', 'N', 'X')
('B', 'I', 'R', 'W')
('C', 'L', 'T', 'V')
('D', 'M', 'P', 'Y')
('E', 'H', 'O', 'U')
('F', 'K', 'S', 'Z')
